In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
df_1 = pd.read_csv("homework_8.1.csv")
df_1.head(5)

,Unnamed: 0,X,Y,Z
0,0,1,4.109218,1.764052
1,1,0,2.259504,0.400157
2,2,0,-0.647584,0.978738
3,3,0,2.106071,2.240893
4,4,1,3.583464,1.867558


In [7]:
# Extract variables
X_treatment = df_1["X"]
Y_outcome = df_1["Y"]
Z_covariate = df_1[["Z"]]

# Fit logistic regression to estimate propensity scores
log_reg = LogisticRegression()
log_reg.fit(Z_covariate, X_treatment)

# Predict propensity scores
propensity_scores = log_reg.predict_proba(Z_covariate)[:,1]

# Compute inverse probability weights
weights = np.where(
    X_treatment == 1,
    1 / propensity_scores,
    1 / (1 - propensity_scores)
)

# Compute weighted mean outcomes for treated and untreated
treated_mask = X_treatment == 1
untreated_mask = X_treatment == 0

# Weighted mean Y for treated
mean_Y_treated = np.sum(Y_outcome[treated_mask] * weights[treated_mask]) / np.sum(weights[treated_mask])
mean_Y_untreated = np.sum(Y_outcome[untreated_mask] * weights[untreated_mask]) / np.sum(weights[untreated_mask])

# Caluculate ATE
ATE = mean_Y_treated - mean_Y_untreated
print(f"ATE is {ATE}")

ATE is 2.2743411898510133


In [8]:
propensity_scores[:3]

array([0.84011371, 0.58464597, 0.71108245])

In [9]:
df_2 = pd.read_csv("homework_8.2.csv")
df_2.head(5)

,Unnamed: 0,X,Y,Z1,Z2
0,0,1,4.652085,1.764052,2.320015
1,1,1,2.590221,0.400157,1.292631
2,2,1,3.898981,0.978738,0.556423
3,3,1,5.857179,2.240893,2.345607
4,4,1,3.647489,1.867558,2.095611


In [11]:
from scipy.spatial import distance

In [ ]:
# Separate treated and untreated group
treated = df_2[df_2["X"] == 1].reset_index(drop=True)
untreated = df_2[df_2["X"] == 0].reset_index(drop=True)

# Z matrices
Z_treated = treated[["Z1", "Z2"]].values
Z_untreated = untreated[["Z1", "Z2"]].values

# Covariance and inverse covariance
Z_all = df_2[["Z1", "Z2"]].values
cov_matrix = np.cov(Z_all, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# compute all pairwise Mahalanobis distances
dists_matrix = distance.cdist(Z_treated, Z_untreated, metric='mahalanobis', VI=inv_cov_matrix)

# For each treated , get the index of the nearest untreated
nearest_indices = np.argmin(dists_matrix, axis=1)

# Outcomes
Y_treated = treated["Y"].values
Y_untreated_matched = untreated.iloc[nearest_indices]["Y"].values

# Compute ATE
ATE = np.mean(Y_treated - Y_untreated_matched)

print(f"ATE is {ATE}")

ATE is 3.4376789979126094


In [ ]:
# Find the maximum minimum distance 

# For each treated observation, get the minimum distance to any untreated
min_dists_per_treated = np.min(dists_matrix, axis=1)

# Index of the treated observation with the largest of these minimum distances
idx_least_common_support = np.argmax(min_dists_per_treated)

# Get the Z1 and Z2 of this treated observation
z1_z2_farthest_treated = Z_treated[idx_least_common_support]

# Also find the Z1 and Z2 of its nearest untreated match
nearest_idx_for_farthest = nearest_indices[idx_least_common_support]
z1_z2_nearest_untreated = Z_untreated[nearest_idx_for_farthest]

(z1_z2_farthest_treated, z1_z2_nearest_untreated)

(array([2.69622405, 0.53815549]), array([ 1.51999486, -1.28220794]))